In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# E2E ML on GCP: MLOps stage 1 : data management

<table align="left">
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/ml_ops_stage1/mlops_data_management.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/ai/platform/notebooks/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/ml_ops_stage1/mlops_data_management.ipynb">
      Open in Google Cloud Notebooks
    </a>
  </td>
</table>
<br/><br/><br/>

## Overview


This tutorial demonstrates how to use Vertex AI for E2E MLOps on Google Cloud in production. This tutorial covers stage 1 : data management.

### Dataset

The dataset used for this tutorial is the [Chicago Taxi](https://www.kaggle.com/chicago/chicago-taxi-trips-bq). The version of the dataset you will use in this tutorial is stored in a public BigQuery table. The trained model predicts whether someone would leave a tip for a taxi fare.

### Objective

In this tutorial, you create a MLOps stage 1: data management process.

This tutorial uses the following Vertex AI and Data Analytics services:

- `Vertex Datasets`
- `BigQuery`
- `Dataflow`

The steps performed include:

- Explore and visualize the data.
- Create a Vertex `Dataset` resource from `BigQuery` table -- for AutoML training.
- Extract a copy of the dataset to a CSV file in Cloud Storage.
- Create a Vertex `Dataset` resource from CSV files -- alternative for AutoML training.
- Read a sample of the `BigQuery` dataset into a dataframe.
- Generate statistics and data schema using TensorFlow Data Validation from the samples in the dataframe.
- Generate a TFRecord feature specification using TensorFlow Data Validation from the data schema.
- Preprocess a portion of the BigQuery data using `Dataflow` -- for custom training.

### Recommendations

When doing E2E MLOps on Google Cloud, the following best practices with structured (tabular) data are recommended:

 - For large amounts of data, use BigQuery table. Otherwise, use a CSV file stored in Cloud Storage.
 - When storing a large amount of data in CSV file, shard the data at 10,000 rows per shard.
 - Create a managed dataset with Vertex `TabularDataset`.
 - Preprocess the data with `Dataflow`

## Installation

Install the latest version of Vertex SDK for Python.

In [ ]:
import os


# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = '--user'
else:
    USER_FLAG = ''

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG

Install the latest GA version of *Dataflow* library as well.

In [ ]:
! pip3 install -U apache-beam[gcp] $USER_FLAG

Install the latest GA version of *TensorFlow Data Validation* library as well.

In [ ]:
! pip3 install -U tensorflow-data-validation $USER_FLAG

Install the latest GA version of *TensorFlow Transform* library as well.

In [ ]:
! pip3 install -U tensorflow-transform $USER_FLAG

### Restart the kernel

Once you've installed the additional packages, you need to restart the notebook kernel so it can find the packages.

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You may not use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations)

In [ ]:
REGION = 'us-central1'  # @param {type: "string"}

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append the timestamp onto the name of resources you create in this tutorial.

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

When you submit a custom training job using the Vertex SDK, you upload a Python package
containing your training code to a Cloud Storage bucket. Vertex AI runs
the code from this package. In this tutorial, Vertex AI also saves the
trained model that results from your job in the same bucket. You can then
create an `Endpoint` resource based on this output in order to serve
online predictions.

Set the name of your Cloud Storage bucket below. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [ ]:
! gsutil ls -al $BUCKET_NAME

### Set up variables

Next, set up some variables used throughout the tutorial.
### Import libraries and define constants

In [ ]:
import google.cloud.aiplatform as aip

#### Import BigQuery

Import the BigQuery package into your Python environment.

In [ ]:
from google.cloud import bigquery

#### Import Apache Beam

Import the Apache Beam package into your Python environment.

In [ ]:
import apache_beam as beam

#### Import TFDV

Import the TensorFlow Data Validation (TFDV) package into your Python environment.

In [ ]:
import tensorflow_data_validation as tfdv

#### Import TensorFlow Transform

Import the TensorFlow Transform (TFT) package into your Python environment.

In [ ]:
import tensorflow_transform as tft

### Initialize Vertex SDK for Python

Initialize the Vertex SDK for Python for your project and corresponding bucket.

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_NAME)

### Create BigQuery client

Create the BigQuery client.

In [ ]:
bqclient = bigquery.Client()

## Datasets

Next, you look at options for creating managed datasets:

* `BigQuery`: Create a Vertex `TabularDataset` resource.
* `CSV`: Create a Vertex `TabularDataset` resource.
* `TFRecords`: Self-manage the dataset on Cloud Storage storage.

#### Location of BigQuery training data.

Now set the variable `IMPORT_FILE` to the location of the data table in BigQuery.

In [ ]:
IMPORT_FILE = 'bq://bigquery-public-data.chicago_taxi_trips.taxi_trips'
BQ_TABLE = 'bigquery-public-data.chicago_taxi_trips.taxi_trips'

### Explore BigQuery dataset

Explore the contents of the BigQuery table:

- Get all examples from 2015
- Sort by the day of the week
- Count the number of examples for each day of the week

In [ ]:
query = """SELECT
    CAST(EXTRACT(DAYOFWEEK FROM trip_start_timestamp) AS string) AS trip_dayofweek,
    FORMAT_DATE('%A',cast(trip_start_timestamp as date)) AS trip_dayname,
    COUNT(*) as trip_count,
FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
WHERE
    EXTRACT(YEAR FROM trip_start_timestamp) = 2015
GROUP BY
    trip_dayofweek,
    trip_dayname
ORDER BY
    trip_dayofweek"""

_ = bqclient.query(query)
rows = _.result()
dataframe = rows.to_dataframe()
print(dataframe.head(7))

In [ ]:
dataframe.plot(kind='bar', x='trip_dayname', y='trip_count')

### Make a private copy of subset of BigQuery table

Next, you make a private copy of the BigQuery table:
- Select a subset of columns
- Select a subset of rows (LIMIT)
- Set conditions (WHERE)
- Do feature engineering on geolocation coords.
- Pre-split the dataset

In [ ]:
BQ_DATASET = BQ_TABLE.split('.')[1]
BQ_TABLE_COPY = f"{PROJECT_ID}.{BQ_DATASET}.taxi_trips"
LIMIT = 1000000
YEAR = 2020

query = f'''
CREATE OR REPLACE TABLE `{BQ_TABLE_COPY}`
AS (
    WITH
      taxitrips AS (
      SELECT
        trip_start_timestamp,
        trip_seconds,
        trip_miles,
        payment_type,
        pickup_longitude,
        pickup_latitude,
        dropoff_longitude,
        dropoff_latitude,
        tips,
        fare
      FROM
        `{BQ_TABLE}`
      WHERE pickup_longitude IS NOT NULL
      AND pickup_latitude IS NOT NULL
      AND dropoff_longitude IS NOT NULL
      AND dropoff_latitude IS NOT NULL
      AND trip_miles > 0
      AND trip_seconds > 0
      AND fare > 0
      AND EXTRACT(YEAR FROM trip_start_timestamp) = {YEAR}
    )

    SELECT
      EXTRACT(MONTH from trip_start_timestamp) as trip_month,
      EXTRACT(DAY from trip_start_timestamp) as trip_day,
      EXTRACT(DAYOFWEEK from trip_start_timestamp) as trip_day_of_week,
      EXTRACT(HOUR from trip_start_timestamp) as trip_hour,
      trip_seconds,
      trip_miles,
      payment_type,
      ST_AsText(
          ST_SnapToGrid(ST_GeogPoint(pickup_longitude, pickup_latitude), 0.1)
      ) AS pickup_grid,
      ST_AsText(
          ST_SnapToGrid(ST_GeogPoint(dropoff_longitude, dropoff_latitude), 0.1)
      ) AS dropoff_grid,
      ST_Distance(
          ST_GeogPoint(pickup_longitude, pickup_latitude),
          ST_GeogPoint(dropoff_longitude, dropoff_latitude)
      ) AS euclidean,
      CONCAT(
          ST_AsText(ST_SnapToGrid(ST_GeogPoint(pickup_longitude,
              pickup_latitude), 0.1)),
          ST_AsText(ST_SnapToGrid(ST_GeogPoint(dropoff_longitude,
              dropoff_latitude), 0.1))
      ) AS loc_cross,
      IF((tips/fare >= 0.2), 1, 0) AS tip_bin,
    FROM
      taxitrips
    LIMIT {LIMIT}
)
'''

response = bqclient.query(query)
_ = response.result()

BQ_TABLE = BQ_TABLE_COPY
IMPORT_FILE = f"bq://{BQ_TABLE_COPY}"

### Create the Dataset

#### BigQuery input data

Next, create the `Dataset` resource using the `create` method for the `TabularDataset` class, which takes the following parameters:

- `display_name`: The human readable name for the `Dataset` resource.
- `bq_source`: Import data items from a BigQuery table into the `Dataset` resource.

Learn more about [TabularDataset from BigQuery table](https://cloud.google.com/vertex-ai/docs/datasets/create-dataset-api#aiplatform_create_dataset_tabular_bigquery_sample-python).

In [ ]:
dataset = aip.TabularDataset.create(
    display_name="Chicago Taxi" + "_" + TIMESTAMP,
    bq_source=[IMPORT_FILE]
)

label_column="tip_bin"

print(dataset.resource_name)

### Copy the dataset to Cloud Storage

Next, you make a copy of the BigQuery dataset, as a CSV file, to Cloud Storage using the BigQuery extract command.

Learn more about [BigQuery command line interface](https://cloud.google.com/bigquery/docs/reference/bq-cli-reference).

In [ ]:
comps = BQ_TABLE.split('.')
BQ_PROJECT_DATASET_TABLE = comps[0] + ':' + comps[1] + '.' + comps[2]

! bq --location=us extract --destination_format CSV $BQ_PROJECT_DATASET_TABLE $BUCKET_NAME/mydata*.csv

IMPORT_FILES = ! gsutil ls $BUCKET_NAME/mydata*.csv

print(IMPORT_FILES)

EXAMPLE_FILE = IMPORT_FILES[0]

! gsutil cat $EXAMPLE_FILE | head

### Create the Dataset

#### CSV input data

Next, create the `Dataset` resource using the `create` method for the `TabularDataset` class, which takes the following parameters:

- `display_name`: The human readable name for the `Dataset` resource.
- `gcs_source`: A list of one or more dataset index files to import the data items into the `Dataset` resource.

Learn more about [TabularDataset from CSV files](https://cloud.google.com/vertex-ai/docs/datasets/create-dataset-api#aiplatform_create_dataset_tabular_gcs_sample-python)

In [ ]:
if 'IMPORT_FILES' in globals():
    gcs_source = IMPORT_FILES
else:
    gcs_source = [IMPORT_FILE]

dataset = aip.TabularDataset.create(
    display_name="Chicago Taxi" + "_" + TIMESTAMP,
    gcs_source=gcs_source
)


label_column="tip_bin"

print(dataset.resource_name)

### Read the BigQuery dataset into a pandas dataframe

Next, you read a sample of the dataset into a pandas dataframe using BigQuery `list_rows()` and `to_dataframe()` method, as follows:

- `list_rows()`: Performs a query on the specified table and returns a row iterator to the query results. Optionally specify:
 - `selected_fields`: Subset of fields (columns) to return.
 - `max_results`: The maximum number of rows to return. Same as SQL LIMIT command.


- `rows.to_dataframe()`: Invokes the row iterator and reads in the data into a pandas dataframe.

Learn more about [Loading BigQuery table into a dataframe](https://cloud.google.com/bigquery/docs/bigquery-storage-python-pandas)

In [ ]:
# Download a table.
table = bigquery.TableReference.from_string(
    BQ_TABLE
)

rows = bqclient.list_rows(
    table,
    max_results=2000

)

dataframe = rows.to_dataframe()
print(dataframe.head())

###  Generate dataset statistics

#### Dataframe input data

Generate statistics on the dataset with the TensorFlow Data Validation (TFDV) package. Use the `generate_statistics_from_dataframe()` method, with the following parameters:

- `dataframe`: The dataset in an in-memory pandas dataframe.
- `stats_options`: The selected statistics options:
  - `label_feature`: The column which is the label to predict.
  - `sample_rate`: The sampling rate. If specified, statistics is computed over the sample.
  - `num_top_values`: number of most frequent feature values to keep for string features.

Learn about [TensorFlow Data Validation (TFDV)](https://www.tensorflow.org/tfx/data_validation/get_started).

In [ ]:
stats = tfdv.generate_statistics_from_dataframe(
    dataframe=dataframe,
    stats_options=tfdv.StatsOptions(
        label_feature='tip_bin',
        sample_rate=1,
        num_top_values=50
    )
)

print(stats)

### Visualize dataset statistics

A visualization of the dataset statistics can be displayed using the TFDV `visualize_statistics()` method.

In [ ]:
tfdv.visualize_statistics(stats)

###  Generate dataset statistics

#### CSV input data

Generate statistics on the dataset with the TensorFlow Data Validation (TFDV) package. Use the `generate_statistics_from_csv()` method, with the following parameters:

- `data_location`: The dataset Cloud Storage file location.
- `stats_options`: The selected statistics options:
  - `label_feature`: The column which is the label to predict.
  - `num_top_values`: number of most frequent feature values to keep for string features.

Learn about [TensorFlow Data Validation (TFDV)](https://www.tensorflow.org/tfx/data_validation/get_started).

In [ ]:
stats = tfdv.generate_statistics_from_csv(
    data_location=EXAMPLE_FILE,
    stats_options=tfdv.StatsOptions(
        label_feature='tip_bin',
        num_top_values=50
    )
)

print(stats)

###  Generate the raw data schema

Generate the data schema on the dataset with the TensorFlow Data Validation (TFDV) package. Use the `infer_schema()` method, with the following parameters:

- `statistics`: The statistics generated by TFDV.

In [ ]:
schema = tfdv.infer_schema(statistics=stats)
print(schema)

In [ ]:
import os

SCHEMA_LOCATION = os.path.join(BUCKET_NAME, "schema.txt")
tfdv.write_schema_text(output_path=SCHEMA_LOCATION, schema=schema)

### Generate the feature specification

Generate the feature specification, compatible with TFRecords, on the dataset with the TensorFlow Transform (TFT) package. Use the `schema_as_feature_spec()` method, with the following parameters:

- `schema`: The data schema generated by TFDV.

In [ ]:
feature_spec = tft.tf_metadata.schema_utils.schema_as_feature_spec(
        schema
    ).feature_spec

print(feature_spec)

### Preprocess data with Dataflow

#### Data Preprocessing

Next, you preprocess the data using Dataflow. In this example, you query the BigQuery table and split the examples into training and evaluation datasets and preprocess feature columns:

- `Numeric`: Rescale the values with `tft.scale_to_z_score`.
- `Categorical`: Encode as a categorical column with `tft.compute_and_apply_vocabulary`.

For expendiency, the number of examples from the dataset is limited to 500.

In [ ]:
import os
import tensorflow_transform.beam as tft_beam
import tensorflow as tf

NUMERIC_FEATURES=["trip_miles", "euclidean"]
CAT_FEATURES=["payment_type", "pickup_grid", "dropoff_grid", "loc_cross"]

def parse_bq_record(bq_record):
    """Parses a bq_record to a dictionary."""
    output = {}
    for key in bq_record:
        output[key] = [bq_record[key]]
    return output


def split_dataset(bq_row, num_partitions, ratio):
    """Returns a partition number for a given bq_row."""
    import json

    assert num_partitions == len(ratio)
    bucket = sum(map(ord, json.dumps(bq_row))) % sum(ratio)
    total = 0
    for i, part in enumerate(ratio):
        total += part
        if bucket < total:
            return i
    return len(ratio) - 1

def preprocessing_fn(inputs):
    outputs = {}
    for key in inputs.keys():
        if key in NUMERIC_FEATURES:
            outputs[key] = tft.scale_to_z_score(inputs[key])
        elif key in CAT_FEATURES:
            outputs[key] = tft.compute_and_apply_vocabulary(
                inputs[key],
                num_oov_buckets=1,
                vocab_filename=key,
            )
        else:
            outputs[key] = inputs[key]
        outputs[key] = tf.squeeze(outputs[key], -1)
    return outputs

def run_pipeline(args):
    """Runs a Beam pipeline to split the dataset"""

    pipeline_options = beam.pipeline.PipelineOptions(flags=[], **args)

    raw_data_query = args["raw_data_query"]
    exported_data_prefix = args["exported_data_prefix"]
    transformed_data_prefix = args["transformed_data_prefix"]
    transform_artifact_dir = args["transform_artifact_dir"]
    temp_location = args["temp_location"]
    project = args["project"]

    schema = tfdv.load_schema_text(SCHEMA_LOCATION)
    feature_spec = tft.tf_metadata.schema_utils.schema_as_feature_spec(
        schema
    ).feature_spec

    raw_metadata = tft.tf_metadata.dataset_metadata.DatasetMetadata(
        tft.tf_metadata.schema_utils.schema_from_feature_spec(feature_spec)
    )

    with beam.Pipeline(options=pipeline_options) as pipeline:
        with tft_beam.Context(temp_location):

            # Read raw BigQuery data.
            raw_train_data, raw_eval_data = (
                pipeline
                | "Read Raw Data"
                >> beam.io.ReadFromBigQuery(
                    query=raw_data_query,
                    project=project,
                    use_standard_sql=True,
                )
                | "Parse Data" >> beam.Map(parse_bq_record)
                | "Split" >> beam.Partition(split_dataset, 2, ratio=[8, 2])
            )

            # Create a train_dataset from the data and schema.
            raw_train_dataset = (raw_train_data, raw_metadata)

            # Analyze and transform raw_train_dataset to produced transformed_train_dataset and transform_fn.
            transformed_train_dataset, transform_fn = (
                raw_train_dataset
                | "Analyze & Transform"
                >> tft_beam.AnalyzeAndTransformDataset(preprocessing_fn)
            )

            # Get data and schema separately from the transformed_dataset.
            transformed_train_data, transformed_metadata = transformed_train_dataset

           # write transformed train data.
            _ = (
                transformed_train_data
                | "Write Transformed Train Data"
                >> beam.io.tfrecordio.WriteToTFRecord(
                    file_path_prefix=os.path.join(
                        transformed_data_prefix, "train/data"
                    ),
                    file_name_suffix=".gz",
                    coder=tft.coders.ExampleProtoCoder(transformed_metadata.schema),
                )
            )

            # Create a eval_dataset from the data and schema.
            raw_eval_dataset = (raw_eval_data, raw_metadata)

            # Transform raw_eval_dataset to produced transformed_eval_dataset using transform_fn.
            transformed_eval_dataset = (
                raw_eval_dataset,
                transform_fn,
            ) | "Transform" >> tft_beam.TransformDataset()

            # Get data from the transformed_eval_dataset.
            transformed_eval_data, _ = transformed_eval_dataset

            # write transformed eval data.
            _ = (
                transformed_eval_data
                | "Write Transformed Eval Data"
                >> beam.io.tfrecordio.WriteToTFRecord(
                    file_path_prefix=os.path.join(transformed_data_prefix, "eval/data"),
                    file_name_suffix=".gz",
                    coder=tft.coders.ExampleProtoCoder(transformed_metadata.schema),
                )
            )

            # Write transform_fn.
            _ = transform_fn | "Write Transform Artifacts" >> tft_beam.WriteTransformFn(
                transform_artifact_dir
            )


EXPORTED_DATA_PREFIX = os.path.join(BUCKET_NAME, 'exported_data')
TRANSFORMED_DATA_PREFIX = os.path.join(BUCKET_NAME, 'transformed_data')
TRANSFORM_ARTIFACTS_DIR = os.path.join(BUCKET_NAME, 'transformed_artifacts')

QUERY_STRING = "SELECT * FROM {0} LIMIT 500".format(BQ_TABLE)
JOB_NAME = "chicago" + TIMESTAMP

args = {
    'runner': 'DirectRunner',
    'raw_data_query': QUERY_STRING,
    'exported_data_prefix': EXPORTED_DATA_PREFIX,
    'transformed_data_prefix': TRANSFORMED_DATA_PREFIX,
    'transform_artifact_dir': TRANSFORM_ARTIFACTS_DIR,
    'temp_location': os.path.join(BUCKET_NAME, 'temp'),
    'project': PROJECT_ID
}

print('Data preprocessing started...')
run_pipeline(args)
print('Data preprocessing completed.')

! gsutil ls $TRANSFORMED_DATA_PREFIX/train
! gsutil ls $TRANSFORMED_DATA_PREFIX/eval
! gsutil ls $TRANSFORM_ARTIFACTS_DIR

# Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

- Dataset
- Pipeline
- Model
- Endpoint
- AutoML Training Job
- Batch Job
- Custom Job
- Hyperparameter Tuning Job
- Cloud Storage Bucket

In [ ]:
delete_all = True

if delete_all:
    # Delete the dataset using the Vertex dataset object
    try:
        if 'dataset' in globals():
            dataset.delete()
    except Exception as e:
        print(e)

    # Delete the model using the Vertex model object
    try:
        if 'model' in globals():
            model.delete()
    except Exception as e:
        print(e)

    # Delete the endpoint using the Vertex endpoint object
    try:
        if 'endpoint' in globals():
            endpoint.delete()
    except Exception as e:
        print(e)

    # Delete the AutoML or Pipeline trainig job
    try:
        if 'dag' in globals():
            dag.delete()
    except Exception as e:
        print(e)

    # Delete the custom trainig job
    try:
        if 'job' in globals():
            job.delete()
    except Exception as e:
        print(e)

    # Delete the batch prediction job using the Vertex batch prediction object
    try:
        if 'batch_predict_job' in globals():
            batch_predict_job.delete()
    except Exception as e:
        print(e)

    # Delete the hyperparameter tuning job using the Vertex hyperparameter tuning object
    try:
        if 'hpt_job' in globals():
            hpt_job.delete()
    except Exception as e:
        print(e)

    if 'BUCKET_NAME' in globals():
        ! gsutil rm -r $BUCKET_NAME